In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Reload libraries automatically
%load_ext autoreload
%autoreload 2

# Plot in the notebook
%matplotlib inline

In [2]:
import lib.model.model as model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
# Get the traits for the agent 1
# Get agents traits
agent='1'
agents = model.get_agents(root_folder="../") 
# Agents
try:
    agent_traits = agents[agent]
except:
    print("Problems retrieving the traits of agent {} in the JSON.".format(agent))
    pprint(agents, indent=3)
    sys.exit(666)
# validation_f = '../data/validation/'
# agent1 = pd.read_csv(validation_f+'validation_agent_1.csv')

In [4]:
agent1 = agent_traits
agent1['nf_ko']

0.1

In [5]:
# Get validation data set
validation_f = '../data/validation/'
data_a1 = pd.read_csv(validation_f+'validation_agent_1.csv')
data_a1.head()

,cat_con,cat_edu,cat_ent,cat_new,cat_per,msg_com,msg_med,msg_qua,msg_que,msg_rel,msg_sal,msg_sen,mood,like,share,comment
0,0,0,0,1,0,0.0,0.5,0.5,0,1,1.0,1.0,0.50,1,1,0
1,0,0,0,1,0,0.0,0.5,0.5,0,1,1.0,1.0,0.51,1,1,0
2,0,0,0,1,0,0.0,0.5,0.5,0,1,1.0,1.0,0.52,1,1,0
3,0,0,0,1,0,0.0,0.5,0.5,0,1,1.0,1.0,0.53,1,1,0
4,0,0,0,1,0,0.0,0.5,0.5,0,1,1.0,1.0,0.54,1,1,0


In [6]:
messages = data_a1[['cat_per', 'cat_ent', 'cat_new', 'cat_edu', 
                    'cat_con', 'msg_rel', 'msg_qua', 'msg_sen', 
                    'msg_sal', 'msg_med', 'msg_com', 'msg_que']]

In [7]:
reactions = data_a1[['mood', 'like', 'comment', 'share']]

In [8]:
messages.shape

(250, 12)

In [9]:
sum_err = 0

In [10]:
# Fitting the empirical data for a1 (reactions)

reactions = reactions.rename(columns=lambda x: x.strip())
reactions['time'] = (reactions.index+1)*20-1

In [11]:
reactions.index = reactions['time']
reactions = reactions[['mood','like', 'comment', 'share']]

In [12]:
reactions.index

Int64Index([  19,   39,   59,   79,   99,  119,  139,  159,  179,  199,
            ...
            4819, 4839, 4859, 4879, 4899, 4919, 4939, 4959, 4979, 4999],
           dtype='int64', name='time', length=250)

### Run 

In [13]:
# Agent
df1, parameters = model.run_message_sequence(messages, agent1, title='nb1', root_folder="../")

In [14]:
parameters

{'srs_sal': [0.7, 2],
 'arousal': [0.9, 2],
 'attention_1': [2.8, 2],
 'attention_2': [0.5, 5],
 'mood': [1.4, 2],
 'ff_ko': [1.4, 2],
 'ff_ent': [1.4, 2],
 'ff_si': [1.4, 4],
 'ff_is': [2.1, 4],
 'ff_se': [2.8, 2],
 'satisfaction': [1.0, 5],
 'prep_like': [2.8, 2],
 'prep_comm': [3.5, 2],
 'prep_share': [2.1, 3],
 'mood_speed': 0.01,
 'prep_gen': [2.5, 2]}

In [15]:
df1[['mood', 'like', 'share', 'comment']].tail()

,mood,like,share,comment
4995,0.010254,0.005495,0.005384,0.001383
4996,0.010254,0.005495,0.005384,0.001383
4997,0.010254,0.005495,0.005384,0.001383
4998,0.010254,0.005495,0.005384,0.001383
4999,0.010255,0.005495,0.005384,0.001383


In [16]:
df1.index = df1.index.astype(int)

In [17]:
dfpoints1 = df1[['mood', 'like', 'share', 'comment']].iloc[reactions.index]

In [18]:
#dfpoints1.index.name = 'time'
dfpoints1.head()

,mood,like,share,comment
19,0.429644,0.006632,0.006762,0.001674
39,0.369336,0.006330,0.006388,0.001596
59,0.317180,0.006070,0.006071,0.001530
79,0.272231,0.005850,0.005806,0.001474
99,0.233603,0.005663,0.005583,0.001426


In [19]:
reactions.head()

,mood,like,comment,share
time,,,,
19,0.50,1,0,1
39,0.51,1,0,1
59,0.52,1,0,1
79,0.53,1,0,1
99,0.54,1,0,1


In [20]:
(abs(dfpoints1-reactions)).sum().sum()

556.029416529291

In [21]:
((dfpoints1-reactions)**2).sum().sum()

517.592951754311

In [22]:
sum_err = sum_err + ((dfpoints1-reactions)**2).sum().sum()

In [23]:
sum_err

517.592951754311

In [24]:
dfpoints1-reactions

,comment,like,mood,share
19,0.001674,-0.993368,-0.070356,-0.993238
39,0.001596,-0.993670,-0.140664,-0.993612
59,0.001530,-0.993930,-0.202820,-0.993929
79,0.001474,-0.994150,-0.257769,-0.994194
99,0.001426,-0.994337,-0.306397,-0.994417
119,0.001386,-0.994493,-0.349515,-0.994602
139,0.001353,-0.994625,-0.387853,-0.994756
159,0.001325,-0.994735,-0.422063,-0.994885
179,0.001301,-0.994828,-0.452719,-0.994993
199,0.001281,-0.994906,-0.480323,-0.995082


In [25]:
inputsDF, parameters = model.run_message_sequence(message_seq=msg_sequence, traits=np.random.rand(7))

NameError: name 'msg_sequence' is not defined